In [62]:
import numpy as np
import re
from collections import Counter
from joblib import dump, load

In [2]:
import gensim
import numpy as np
import contractions
from collections import Counter
from nltk.corpus import stopwords
from statistics import mean,stdev
from num2words import num2words
from spellchecker import SpellChecker
spell = SpellChecker()
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
stop_words = stopwords.words('english')
tokenizer = RegexpTokenizer('[A-Za-z0-9]*[.]?\w+')

In [3]:
dataset_with_labels = load('dataset_with_labels.sav')

In [4]:
len(dataset_with_labels)

11112

In [5]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('en_wiki_word2vec_300.txt', binary=False) 

In [6]:
w2v_vocab = set(w2v_model.wv.vocab)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [63]:
def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def word_prob(word): 
    return dictionary[word] / total

def words(text): 
    return re.findall('[a-z]+', text.lower()) 

dictionary = Counter(w2v_vocab)
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

In [65]:
def get_num2word_tokens(token):
    tokens = []
    if token.isdecimal():
        token = int(token)
    else:
        token = float(token)
    t = num2words(token)
    t = tokenizer.tokenize(t)
    for tok in t:
        if tok not in stop_words:
            tokens.append(tok)
    return tokens

In [66]:
invalids1, invalids2 = [], []

In [67]:
def validate_alphab_word(tok, w2v_vocab):
    if tok == 'a':
        return ['an']
    elif tok == 'i':
        return ['you']
    spell_checked = spell.correction(tok).lower()
    tokens = tokenizer.tokenize(spell_checked)
    if len(tokens) >1 or tokens[0] in w2v_vocab:
#         print('spelling', tokens)
        return tokens
    token = lemmatizer.lemmatize(tokens[0])
    if token in w2v_vocab:
#         print(tok, 'lemmat', token)
        return [token]
    token = stemmer.stem(token)
    if token in w2v_vocab:
#         print(tok, 'stemming', token)
        return [token]
    tokens = viterbi_segment(token)[0]
    if all(t in w2v_vocab for t in tokens):
#         print(tokens)
#         invalids1.append(tokens)
        return tokens
#     invalids2.append(tok)
    return []

In [68]:
def validate_num_word(token):
    token2 = None
    if token[-1] in ['s','m','k']:
        token = token[:-1]
    elif token[-2:] in ['th','nd','st','rd']:
        token = token[:-2]
    elif token[-2:] in ['am','pm','bn','km','ft','mm','mg']:
        token2 = token[-2:]
        token = token[:-2]
    else:
#         print('dropped', token)
        return []
    
    if all(c in "0123456789." for c in token) and token.count('.') <= 1:
        t = get_num2word_tokens(token)
        if token2 is not None:
            t.append(token2)
        return t
    else:
#         print('dropped', token)
        return []

In [69]:
def normalize(doc, w2v_vocab):
    doc = contractions.fix(doc)
    if ',' in doc:
        doc = doc.replace(",", "")
    tokens = tokenizer.tokenize(doc)
    normalized_tokens = []
    for token in tokens:
        token = token.lower()
        if len(token) <= 1 and token not in ['a', 'i']:
            continue
        if all(c in "0123456789." for c in token) and token.count('.') <= 1:
            t = get_num2word_tokens(token)
            normalized_tokens += t
            continue
        elif any(c in "0123456789" for c in token):
            if token not in w2v_vocab:
                t = validate_num_word(token)
                normalized_tokens += t
                continue
        else:
            token = token.replace(".", "")
            if token not in w2v_vocab:
                t = validate_alphab_word(token, w2v_vocab)
                normalized_tokens += t
                continue
        normalized_tokens.append(token)
    return normalized_tokens

In [35]:
total_tok = []
for x in dataset_with_labels:
    tokens = normalize(x[0],w2v_vocab)
    total_tok += tokens

['desert', 'if']
['desert', 'if']
spelling ['announced']
spelling ['merest']
spelling ['debate']
spelling ['debate']
spelling ['debate']
spelling ['debate']
spelling ['debate']
['abdo', 'rra', 'zaq']
['deh', 'dadi']
['abdo', 'rra', 'zaq']
spelling ['idea']
['zvati', 'chada']
spelling ['peace']
parliamentarians lemmat parliamentarian
['anti', 'globalist']
['anti', 'globalist']
['anti', 'globalist']
['anti', 'globalist']
spelling ['ramallah']
spelling ['kozloduy']
spelling ['kozloduy']
backpedalled stemming backpedal
['would', 'evelop']
spelling ['banzai']
spelling ['bakufu']
spelling ['antithetical']
['krum', 'hanzl']
spelling ['ayrton']
['abdo', 'rrash', 'id']
['mawa', 'panga']
['mwana', 'nanga']
['woul', 'dialogu']
['tbe', 'ish', 'at']
['tbe', 'ish', 'at']
spelling ['indians']
spelling ['indians']
spelling ['indians']
['qalqili', 'yah']
['hydrometeor', 'olog']
['kuz', 'vart']
['uig', 'hh', 'ur']
spelling ['banzai']
['woul', 'dialogu']
['woul', 'dialogu']
['environmental', 'ist']
spell

spelling ['recently']
['telecomm', 'un']
['telecomm', 'un']
spelling ['markedly']
['disen', 'franchis']
spelling ['guantanamo']
spelling ['behn']
spelling ['behn']
spelling ['behn']
spelling ['behn']
spelling ['behn']
spelling ['behn']
spelling ['behn']
spelling ['wobbly']
spelling ['maude']
['constit', 'ut']
['motsu', 'enyan']
spelling ['rename']
['unself', 'cons', 'ci']
spelling ['wouldham']
intergovernmental stemming intergovernment
spelling ['matcham']
spelling ['matcham']
['enviro', 'crat']
['astronaut', 'when']
['lisa', 'ius']
spelling ['ricoh']
spelling ['astronaut']
spelling ['athena']
['fue', 'mayor']
['fede', 'camara']
['fue', 'mayor']
spelling ['kazakhstan']
parliamentarians lemmat parliamentarian
spelling ['schiemann']
['alja', 'deeda']
spelling ['athan']
spelling ['athan']
spelling ['countries']
spelling ['chavez']
spelling ['wanted']
['nettles', 'om']
['would', 'etai', 'ne']
spelling ['emm']
spelling ['emm']
spelling ['welshman']
responsibilities lemmat responsibility
['y

In [36]:
invalids1

[['desert', 'if'],
 ['desert', 'if'],
 ['abdo', 'rra', 'zaq'],
 ['deh', 'dadi'],
 ['abdo', 'rra', 'zaq'],
 ['zvati', 'chada'],
 ['anti', 'globalist'],
 ['anti', 'globalist'],
 ['anti', 'globalist'],
 ['anti', 'globalist'],
 ['would', 'evelop'],
 ['krum', 'hanzl'],
 ['abdo', 'rrash', 'id'],
 ['mawa', 'panga'],
 ['mwana', 'nanga'],
 ['woul', 'dialogu'],
 ['tbe', 'ish', 'at'],
 ['tbe', 'ish', 'at'],
 ['qalqili', 'yah'],
 ['hydrometeor', 'olog'],
 ['kuz', 'vart'],
 ['uig', 'hh', 'ur'],
 ['woul', 'dialogu'],
 ['woul', 'dialogu'],
 ['environmental', 'ist'],
 ['irre', 'spons'],
 ['condole', 'eza'],
 ['giav', 'arini'],
 ['giav', 'arini'],
 ['giav', 'arini'],
 ['giav', 'arini'],
 ['giav', 'arini'],
 ['under', 'develop'],
 ['pieter', 'maritzburg'],
 ['pieter', 'maritzburg'],
 ['pieter', 'maritzburg'],
 ['pieter', 'maritzburg'],
 ['stoud', 'mann'],
 ['baiko', 'nor'],
 ['ros', 'aviaco', 'smo'],
 ['haig', 'ner'],
 ['sandu', 'qah'],
 ['unco', 'nstitut'],
 ['yucel', 'en'],
 ['taes', 'upk', 'yok'],
 [

In [37]:
print(len(invalids1))
invalids1 = set(invalids1)
len(invalids1) # 147

222


TypeError: unhashable type: 'list'

In [38]:
invalids2

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'i',
 'a',
 'a',
 'i',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a'

In [40]:
print(len(invalids2))
invalids2 = set(invalids2)
len(invalids2)
invalids2 #5000

4


{'a', 'f', 'i', 'w'}

In [ ]:
# total_tok

In [41]:
print(len(total_tok))
total_tok = set(total_tok)
len(total_tok)

265080


16219

In [42]:
not_found = total_tok - w2v_vocab 

In [43]:
len(not_found)

0

In [44]:
not_found

set()

In [ ]:
# new_spelled = set()
# for word in not_found:
#     c = spell.correction(word)
#     print(word,'  -  ', c)
#     new_spelled.add(c)

In [70]:
word_embeddings_dataset_1 = []

In [71]:
i = 0
for x in dataset_with_labels:
    i += 1
    vectors = []
    tokens = normalize(x[0],w2v_vocab)
    for token in tokens:
        try:
            vec = w2v_model.wv[token]
        except:
            print(i, token)
            continue
        vectors.append(vec)
    word_embeddings_dataset_1.append([tokens, vectors, x[1]])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [73]:
dump(word_embeddings_dataset_1, 'word_embeddings_dataset_1.sav')

['word_embeddings_dataset_1.sav']

In [88]:
doc_wise_dataset_with_lbs = load('doc_wise_dataset_with_lbs.sav')

In [94]:
len(doc_wise_dataset_with_lbs)

535

In [95]:
word_embeddings_dataset_2 = []

In [96]:
d = -1
j = 0
for doc in doc_wise_dataset_with_lbs:
    d += 1
    i = -1
    all_vectors, all_tokens, fact_vectors, fact_tokens = [], [], [], []
    for stat in doc:
        i += 1
        j += 1
        tokens = normalize(stat[0],w2v_vocab)
        if stat[1] == 0:
            for token in tokens:
                try:
                    vec = w2v_model.wv[token]
                except:
                    print(d, i, token)
                    continue
                all_vectors.append(vec)
            all_tokens += tokens + ['.']
            all_vectors.append(np.zeros(300))
        else:
            for token in tokens:
                try:
                    vec = w2v_model.wv[token]
                except:
                    print(d, i, token)
                    continue
                all_vectors.append(vec)
                fact_vectors.append(vec)
            all_tokens += tokens + ['.']
            fact_tokens += tokens + ['.']
            all_vectors.append(np.zeros(300))
            fact_vectors.append(np.zeros(300))
    word_embeddings_dataset_2.append([all_tokens, all_vectors, fact_tokens, fact_vectors])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [103]:
len(word_embeddings_dataset_2[0][3])

256

In [104]:
dump(word_embeddings_dataset_2, 'word_embeddings_dataset_2.sav')

['word_embeddings_dataset_2.sav']